https://www.kaggle.com/code/ryanholbrook/creating-features

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)




/tmp/ipykernel_33/4170113402.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-whitegrid")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd0 in position 0: invalid continuation byte

In [ ]:
accidents = pd.read_csv("/kaggle/input/us-accidents/US_Accidents_March23.csv")

In [ ]:
autos = pd.read_csv("/kaggle/input/automobile-dataset/Automobile_data.csv")

In [ ]:
concrete = pd.read_csv("/kaggle/input/ibm-watson-marketing-customer-value-data/WA_Fn-UseC_-Marketing-Customer-Value-Analysis.csv")


In [ ]:
# Interesting--it appears that there is an error in thie file such that it cannot be read.  
# I'm pretty sure I was able to read it in a different exercise...
customer = pd.read_csv("/kaggle/input/concrete-comprehensive-strength/Concrete_Data.xls")

In [7]:
autos.columns.tolist()

['symboling',
 'normalized-losses',
 'make',
 'fuel-type',
 'aspiration',
 'num-of-doors',
 'body-style',
 'drive-wheels',
 'engine-location',
 'wheel-base',
 'length',
 'width',
 'height',
 'curb-weight',
 'engine-type',
 'num-of-cylinders',
 'engine-size',
 'fuel-system',
 'bore',
 'stroke',
 'compression-ratio',
 'horsepower',
 'peak-rpm',
 'city-mpg',
 'highway-mpg',
 'price']

In [9]:
autos[["stroke", "bore"]].head()

,stroke,bore
0,2.68,3.47
1,2.68,3.47
2,3.47,2.68
3,3.4,3.19
4,3.4,3.19


In [10]:
print(autos.dtypes)

symboling              int64
normalized-losses     object
make                  object
fuel-type             object
aspiration            object
num-of-doors          object
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight            int64
engine-type           object
num-of-cylinders      object
engine-size            int64
fuel-system           object
bore                  object
stroke                object
compression-ratio    float64
horsepower            object
peak-rpm              object
city-mpg               int64
highway-mpg            int64
price                 object
dtype: object


Get the indices where there is an unknown value, which cannot be used in the calculations.

In [22]:
unknown_strokes = autos.index[autos['stroke'] == "?"].tolist()

In [19]:
unknown_bores = autos.index[autos['bore'] == "?"].tolist()

In [28]:
invalid_auto_data = list(set(unknown_bores + unknown_strokes))
print(invalid_auto_data)

[56, 57, 58, 55]


In [29]:
autos = autos.drop(invalid_auto_data)
autos["stroke"] = autos["stroke"].astype(float)
autos["bore"] = autos["bore"].astype(float)

In [31]:
autos["stroke_ratio"] = autos.stroke / autos.bore

print(autos.dtypes)

symboling              int64
normalized-losses     object
make                  object
fuel-type             object
aspiration            object
num-of-doors          object
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight            int64
engine-type           object
num-of-cylinders      object
engine-size            int64
fuel-system           object
bore                 float64
stroke               float64
compression-ratio    float64
horsepower            object
peak-rpm              object
city-mpg               int64
highway-mpg            int64
price                 object
stroke_ratio         float64
dtype: object


In [36]:
autos[["stroke", "bore", "stroke_ratio", "num-of-cylinders"]].head()

,stroke,bore,stroke_ratio,num-of-cylinders
0,2.68,3.47,0.772334,four
1,2.68,3.47,0.772334,four
2,3.47,2.68,1.294776,six
3,3.40,3.19,1.065831,four
4,3.40,3.19,1.065831,five


hah, the cylinders column is strings names of numbers...
https://stackoverflow.com/questions/493174/is-there-a-way-to-convert-number-words-to-integers

In [37]:
!pip install word2number

  Preparing metadata (setup.py) ... done
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5568 sha256=64cdacda2f786b3c484da4d7aba65d9fcbb3bf49a37f72a662f3d2b28ca94508
  Stored in directory: /root/.cache/pip/wheels/84/ff/26/d3cfbd971e96c5aa3737ecfced81628830d7359b55fbb8ca3b
Successfully built word2number


In [41]:
from word2number import w2n

print(w2n.word_to_num(autos["num-of-cylinders"][0]))

4


In [46]:
autos["num-of-cylinders"] = autos.apply(lambda x: w2n.word_to_num(x["num-of-cylinders"]), axis=1)

In [47]:
autos["num-of-cylinders"].head()

0      4
1      4
2      6
3      4
4      5
      ..
200    4
201    4
202    6
203    6
204    4
Name: num-of-cylinders, Length: 201, dtype: int64

In [42]:
# This does not work, see above...
# autos["num-of-cylinders"] = w2n.word_to_num(autos["num-of-cylinders"])

ValueError: Type of input is not string! Please enter a valid number word (eg. 'two million twenty three thousand and forty nine')

In [50]:
autos["displacement"] = (
    np.pi * ((0.5 * autos.bore) ** 2) * autos.stroke * autos["num-of-cylinders"]
)

In [51]:
autos["displacement"].head()

0    101.377976
1    101.377976
2    117.446531
3    108.695147
4    135.868934
Name: displacement, dtype: float64